## 1.测试系统Pytroch, GPU, AIshell数据集可用性

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import torch 
print("GPU status:"+str(torch.cuda.is_available()) )
print("pytorch version:"+torch.__version__)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from IPython.display import Audio
samplefile="/kaggle/input/aishell-1/speech_asr_aishell_trainsets/speech_asr_aishell_trainsets/wav/train/S0002/BAC009S0002W0122.wav"
Audio(samplefile)

! pip install git+https://github.com/openai/whisper.git -q

import whisper

# small_model = whisper.load_model("small")
# medium_model = whisper.load_model("medium")
large_model = whisper.load_model("large")

# small_result = small_model.transcribe(samplefile, fp16=False)
# print(small_result["text"])

# medium_result = medium_model.transcribe(samplefile, fp16=False)
# print(medium_result["text"])

large_result = large_model.transcribe(samplefile, fp16=False)
print(large_result["text"])
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 2.下载例子代码
根据whisper论文，不太强调中文调优，已经训练了足够多的中文音频，所以注释掉aishell代码

In [ ]:
import os
if not os.path.exists("/kaggle/working/Whisper-Finetune"):
    !git clone https://gitee.com/hohoha/Whisper-Finetune.git /kaggle/working/Whisper-Finetune
%cd /kaggle/working/Whisper-Finetune
!git pull 

import shutil
path = "/kaggle/working/modelsA"
if os.path.exists(path):
        shutil.rmtree(path)
        print('删除完成'+path)    

!pip install -r requirements.txt
# %run ./aishell.py --annotation_text /kaggle/working/aishell --target_dir /kaggle/input/aishell-1 --csv_path /kaggle/input/aishell-adaption/speech_asr_aishell_trainsets_utf8.csv


## 3. ctranslate2加速

In [5]:
# !pip install -U whisper-ctranslate2
import faster_whisper
# model = faster_whisper.WhisperModel("whisper-large-v2-ct2")
model_size = "large-v2"
samplefile="/kaggle/input/aishell-1/speech_asr_aishell_trainsets/speech_asr_aishell_trainsets/wav/train/S0002/BAC009S0002W0122.wav"
# Run on GPU with FP16
# model = faster_whisper.WhisperModel(model_size, device="cuda", compute_type="float16",download_root="/kaggle/working/models",local_files_only=False)

# or run on GPU with INT8
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# or run on CPU with INT8
model = faster_whisper.WhisperModel(model_size, device="cpu", compute_type="int8",download_root="/kaggle/working/models",local_files_only=True)
# model = faster_whisper.WhisperModel(model_size, device="cpu", compute_type="int8",download_root="/kaggle/working/models",local_files_only=False)
# model = faster_whisper.WhisperModel("whisper-large-v2-ct2")

segments, info = model.transcribe(samplefile, beam_size=5)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

for segment in segments:
    text = segment.text
    print(f"[{round(segment.start, 2)} - {round(segment.end, 2)}]：{text}\n")

Detected language 'zh' with probability 0.988409
[0.0 - 6.0]：而对面楼是城郊一致作用最大的限购



In [7]:
#打包models
!tar -zcvf /kaggle/working/models.tar.gz /kaggle/working/models

tar: Removing leading `/' from member names
/kaggle/working/models/
/kaggle/working/models/models--guillaumekln--faster-whisper-large-v2/
/kaggle/working/models/models--guillaumekln--faster-whisper-large-v2/blobs/
/kaggle/working/models/models--guillaumekln--faster-whisper-large-v2/blobs/c9074644d9d1205686f16d411564729461324b75
/kaggle/working/models/models--guillaumekln--faster-whisper-large-v2/blobs/bf2a9746382e1aa7ffff6b3a0d137ed9edbd9670c3b87e5d35f5e85e70d0333a
/kaggle/working/models/models--guillaumekln--faster-whisper-large-v2/blobs/7818adb6de9fa3064d3ff81226fdd675be1f6344
/kaggle/working/models/models--guillaumekln--faster-whisper-large-v2/blobs/346bb8684801a61c15ce82bcb80c29a63c2d1f94
/kaggle/working/models/models--guillaumekln--faster-whisper-large-v2/snapshots/
/kaggle/working/models/models--guillaumekln--faster-whisper-large-v2/snapshots/f541c54c566e32dc1fbce16f98df699208837e8b/
/kaggle/working/models/models--guillaumekln--faster-whisper-large-v2/snapshots/f541c54c566e32dc1f

In [11]:
from IPython.display import FileLink
FileLink(r"/kaggle/working/models.tar.gz")

/kaggle/working/models.tar.gz

##4.测试图形界面可行性--需要声卡，不可行

In [ ]:
import os
if not os.path.exists("/kaggle/working/Whisper-Finetune"):
    !git clone https://gitee.com/hohoha/Whisper-Finetune.git /kaggle/working/Whisper-Finetune
%cd /kaggle/working/Whisper-Finetune
!git pull 
!pip install -r requirements.txt
%run ./infer_gui.py

##5.下载模型

In [ ]:
import time
from huggingface_hub import hf_hub_download
repo_id = "guillaumekln/faster-whisper-large-v2" # 仓库ID
local_dir = '/kaggle/working/models'
cache_dir = local_dir + "/cache"
filename= "config.json"
while True:   
    try:
        hf_hub_download(cache_dir=cache_dir,
        local_dir=local_dir,
        repo_id=repo_id,
        filename=filename,
        local_dir_use_symlinks=False,
        resume_download=True,
        etag_timeout=100
        )
    except Exception as e :
        print(e)
        # time.sleep(5)
    else:
        print('下载完成')
        break
